In [1]:
from os import path

# Third-party
from astropy.convolution import convolve, Gaussian1DKernel
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits, ascii
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.optimize import minimize
from scipy.stats import binned_statistic
from astropy.convolution import convolve, Gaussian1DKernel
from astroML.utils import log_multivariate_gaussian

from pyia import GaiaData

import gala.coordinates as gc
import gala.dynamics as gd
from gala.dynamics import mockstream
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
from gala.mpl_style import center_emph

In [2]:
galcen_frame = coord.Galactocentric(galcen_distance=8.1*u.kpc)

## Prepare data:

In [3]:
tbl = Table.read('../output/rrl_bhb_bs_rgb_master.fits')

In [4]:
c = coord.SkyCoord(ra=tbl['ra']*u.deg, 
                   dec=tbl['dec']*u.deg)
c_gd1 = c.transform_to(gc.GD1)

In [7]:
data = Table()

data['phi1'] = c_gd1.phi1.wrap_at(180*u.deg).degree
data['phi1_ivar'] = 1 / (1*u.mas).to_value(u.deg)**2

data['phi2'] = c_gd1.phi2.degree
data['phi2_ivar'] = 1 / (1*u.mas).to_value(u.deg)**2

data['distmod'] = tbl['DM']
data['distmod'][np.isnan(data['distmod'])] = 0.
data['distmod_ivar'] = 1 / tbl['DM_error']**2

data['pmra'] = tbl['pmra']
data['pmra_ivar'] = 1 / tbl['pmra_error']**2

data['pmdec'] = tbl['pmdec']
data['pmdec_ivar'] = 1 / tbl['pmdec_error']**2

data['rv'] = tbl['rv']
data['rv'][np.isnan(data['rv'])] = 0.
data['rv_ivar'] = 1 / tbl['rv_error']**2

data = data.filled(fill_value=0)

INFO: Upgrading Table to masked Table. Use Table.filled() to convert to unmasked table. [astropy.table.table]


---

## HACKING:

In [8]:
_stream = np.load('../output/stream_model_-20.0.npy')
stream = gd.PhaseSpacePosition(pos=_stream[:, :3].T*u.kpc,
                               vel=_stream[:, 3:].T*u.km/u.s)

In [9]:
from scipy.stats import binned_statistic

In [54]:
def get_stream_track(stream_c, stream_icrs,
                     phi1_lim=[-180, 180]*u.deg, 
                     phi1_binsize=1*u.deg):
    
    phi1 = stream_c.phi1.wrap_at(180*u.deg).degree
    phi1_lim = phi1_lim.to_value(u.deg)
    phi1_binsize = phi1_binsize.to_value(u.deg)
    
    phi1_bins = np.arange(phi1_lim[0], phi1_lim[1]+1e-8, phi1_binsize)
    phi1_binc = 0.5*(phi1_bins[:-1]+phi1_bins[1:])
    
    means = dict()
    stds = dict()
    interps = dict()
    units = dict()
    
    # --------
    k = 'phi2'
    val = getattr(stream_c, k)
    means[k] = binned_statistic(phi1, val.value, 
                                bins=phi1_bins, statistic='mean')
    stds[k] = binned_statistic(phi1, val.value, 
                               bins=phi1_bins, statistic='std')
    units[k] = val.unit
    interps[k] = InterpolatedUnivariateSpline(phi1_binc, means[k].statistic, 
                                              w=1/stds[k].statistic**2)
    
    # --------
    k = 'distmod'
    val = getattr(stream_c, 'distance').distmod
    means[k] = binned_statistic(phi1, val.value, 
                                bins=phi1_bins, statistic='mean')
    stds[k] = binned_statistic(phi1, val.value, 
                               bins=phi1_bins, statistic='std')
    units[k] = val.unit
    interps[k] = InterpolatedUnivariateSpline(phi1_binc, means[k].statistic, 
                                              w=1/stds[k].statistic**2)
    
    # --------
    for k, k2 in [('pmra', 'pm_ra_cosdec'), 
                  ('pmdec', 'pm_dec')]:
        val = getattr(stream_icrs, k2)
        means[k] = binned_statistic(phi1, val.value, 
                                    bins=phi1_bins, statistic='mean')
        stds[k] = binned_statistic(phi1, val.value, 
                                   bins=phi1_bins, statistic='std')
        units[k] = val.unit
        interps[k] = InterpolatedUnivariateSpline(phi1_binc, means[k].statistic, 
                                                  w=1/stds[k].statistic**2)
    
    # --------
    k = 'rv'
    val = getattr(stream_c, 'radial_velocity')
    means[k] = binned_statistic(phi1, val.value, 
                                bins=phi1_bins, statistic='mean')
    stds[k] = binned_statistic(phi1, val.value, 
                               bins=phi1_bins, statistic='std')
    units[k] = val.unit
    interps[k] = InterpolatedUnivariateSpline(phi1_binc, means[k].statistic, 
                                              w=1/stds[k].statistic**2)
    
    return interps

In [55]:
def ln_normal(x, mu, std):
    return -0.5 * (x-mu)**2 / std**2 - 0.5*np.log(2*np.pi) - np.log(std)

def ln_normal_ivar(x, mu, ivar):
    return -0.5 * (x-mu)**2 * ivar - 0.5*np.log(2*np.pi) + 0.5*np.log(ivar)

def get_ivar(ivar, extra_var):
    return ivar / (1 + extra_var * ivar)

In [106]:
class MockStreamModel:
    
    @u.quantity_input(phi1_0=u.deg,
                      phi1_lim=u.deg, phi1_binsize=u.deg)
    def __init__(self, data, stream_frame,
                 potential, dt, n_steps, phi1_0,
                 release_every=1,
                 galcen_frame=None,
                 phi1_lim=[-180, 180]*u.deg, phi1_binsize=1*u.deg):
        
        # TODO: validate data?
        self.data = data
        
        # coordinate frame of the stream data
        self.stream_frame = stream_frame
        
        # potential and integration parameters
        self.ham = gp.Hamiltonian(potential)
        self.dt = dt
        self.n_steps = n_steps
        self.phi1_0 = phi1_0
        self.release_every = release_every
        
        if galcen_frame is None:
            galcen_frame = coord.Galactocentric()
        self.galcen_frame = galcen_frame
            
        self.phi1_lim = phi1_lim
        self.phi1_binsize = phi1_binsize
        
        # TODO: put data into the right units? or just strip units and prepare...
    
    def tracks_ln_likelihood(self, stream):
        tracks = get_stream_track(stream.to_coord_frame(self.stream_frame, 
                                                        galactocentric_frame=self.galcen_frame),
                                  stream.to_coord_frame(coord.ICRS, 
                                                        galactocentric_frame=self.galcen_frame),
                                  phi1_lim=self.phi1_lim,
                                  phi1_binsize=self.phi1_binsize)
        
        # _grid = np.linspace(-100, 20, 1024)
        # for k in tracks:
        #     plt.figure(figsize=(10, 5))
        #     plt.scatter(data['phi1'], data[k], zorder=-100)
        #     plt.plot(_grid, tracks[k](_grid), marker='')

        # TODO:
        extra_var = dict()
        extra_var['phi2'] = 0.5 ** 2
        extra_var['distmod'] = 0.02 ** 2
        extra_var['pmra'] = 0.1 ** 2
        extra_var['pmdec'] = 0.1 ** 2
        extra_var['rv'] = 5. ** 2
        
        lls = []
        for name in ['phi2', 'distmod', 'pmra', 'pmdec', 'rv']:
            ll = ln_normal_ivar(tracks[name](data['phi1']), 
                                data[name], 
                                get_ivar(data[name+'_ivar'], extra_var[name]))
            lls.append(ll[np.isfinite(ll)].sum())
        
        return np.sum(lls)
    
    def ln_likelihood(self, p):
        phi2, d, pm1, pm2, rv = p
        
        print(p)
        
        # TODO: vary potential?
        ham = self.ham
        
        c = self.stream_frame.__class__(phi1=self.phi1_0,
                                        phi2=phi2*u.deg,
                                        distance=d*u.kpc,
                                        pm_phi1_cosphi2=pm1*u.mas/u.yr,
                                        pm_phi2=pm2*u.mas/u.yr,
                                        radial_velocity=rv*u.km/u.s,
                                        **self.stream_frame.frame_attributes)
        w0 = gd.PhaseSpacePosition(c.transform_to(self.galcen_frame).data)        
        orbit = ham.integrate_orbit(w0, dt=-np.abs(self.dt), 
                                    n_steps=self.n_steps)[::-1]
        
        # TODO: vary mass-loss!
        n_times = len(orbit.t)
        prog_mass = np.linspace(5e4, 2E3, n_times) * u.Msun
        stream = mockstream.dissolved_fardal_stream(ham, orbit, prog_mass,
                                                    t_disrupt=0*u.Gyr, 
                                                    release_every=self.release_every,
                                                    seed=42)
        
        return self.tracks_ln_likelihood(stream)
    
    def ln_prior(self, p):
        phi2, dm, pm1, pm2, rv = p
        
        lp = 0.
        
        # TODO HACK: hard-coded
        lp += ln_normal(phi2, 0., 3.)
        lp += ln_normal(dm, 14.65, 1.)
        lp += ln_normal(pm1, 0, 50.)
        lp += ln_normal(pm2, 0, 50.)
        lp += ln_normal(rv, 0, 400.)
        
        return lp

In [107]:
model = MockStreamModel(data, stream_frame=gc.GD1(),
                        potential=gp.MilkyWayPotential(), 
                        dt=-0.5*u.Myr, n_steps=8000, phi1_0=-20*u.deg,
                        galcen_frame=galcen_frame,
                        phi1_lim=[-100, 20]*u.deg, phi1_binsize=2*u.deg)

In [108]:
p0 = (4.14772642e-02,  8.90676434e+00, -1.00000000e+01, -2.27057862e+00, -1.37929779e+02)

In [109]:
model.ln_prior(p0)

-36.0817720668013

In [110]:
%%time
model.ln_likelihood(p0)

(0.0414772642, 8.90676434, -10.0, -2.27057862, -137.929779)
CPU times: user 7.6 s, sys: 59.1 ms, total: 7.66 s
Wall time: 7.68 s


/Users/adrian/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log


-2276.5649851265007

In [112]:
res = minimize(lambda *x: -model.ln_likelihood(*x), 
               x0=p0, 
               bounds=[(-2, 2), (5, 12), (-20, 0), 
                       (-10, 10), (-200, -100)])

[ 4.14772642e-02  8.90676434e+00 -1.00000000e+01 -2.27057862e+00
 -1.37929779e+02]


/Users/adrian/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log


[ 4.14772742e-02  8.90676434e+00 -1.00000000e+01 -2.27057862e+00
 -1.37929779e+02]
[ 4.14772642e-02  8.90676435e+00 -1.00000000e+01 -2.27057862e+00
 -1.37929779e+02]
[ 4.14772642e-02  8.90676434e+00 -9.99999999e+00 -2.27057862e+00
 -1.37929779e+02]
[ 4.14772642e-02  8.90676434e+00 -1.00000000e+01 -2.27057861e+00
 -1.37929779e+02]
[ 4.14772642e-02  8.90676434e+00 -1.00000000e+01 -2.27057862e+00
 -1.37929779e+02]
[   2.   12.    0.   10. -100.]


ERROR: TypeError: can't convert complex to float [IPython.core.interactiveshell]


TypeError: can't convert complex to float

In [ ]:
res.x - p0